In [1]:
%matplotlib inline

import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 

ylims=[0.67, 0.45, 0.60, 0.59, 0.38]
all_ylims = [0.67, 0.45, 0.60, 0.75, 0.59, 0.38, 20]

In [2]:
queries = load_queries(config.AUS_TOPIC_PATH)

In [3]:
BASE_DIR = os.path.join(os.environ["HOME"], "phd-generated")

In [4]:
index_name = 'filtered-phrasestop'#, config.SIGIR_INDEX_NAME]
qrel_paths =  [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['AUS']#, 'SIGIR']

In [5]:
base_df = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_query = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

In [6]:
methods = ['idf', 'ictf', 'kli', 'plm']
start = 0 
end = 1.0
inc = 0.1 
_dir = os.path.join(BASE_DIR, 'reduction', 'dirichlet_prior')

#filtered-phrasestop-plm-1.00-topics-filtered-phrasestop-unigram_dir_mu_1050.00
method_runs = [load_1d_dfs([index_name], qrel_paths, _dir, '{0}-'+x+'-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, start, end, inc)[0] for x in methods]


In [7]:
plot = plot_tune_1d_comp(methods, config.METRIC_NAMES, method_runs, start, end, inc, ylims=all_ylims)
plot.savefig('figures/ausnl-reduction.pdf')

<Figure size 1152x432 with 7 Axes>

In [8]:
qry_runs = method_runs = [load_1d_dfs([index_name], qrel_paths, _dir, '{0}-'+x+'-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, start, end, inc, per_query=True)[0] for x in methods]

In [9]:
(qry_runs[-1][-2]-base_query).sort_values('rbp@0.80')[-10:]

,recip_rank,unjudged@20,recall_20,recall_100,ndcg,set_P,set_recall,set_F_.5,rbp@0.10,rbp-res@0.10,rbp@0.50,rbp-res@0.50,rbp@0.80,rbp-res@0.80,err@20
49,0.1667,4.0,0.0000,-0.0556,-0.0484,-0.0100,-0.0556,-0.0137,0.0810,0.0,0.1259,0.0040,0.0468,0.0582,-0.15372
80,0.0873,1.0,0.0000,0.0000,0.0464,0.0000,0.0000,0.0000,0.0000,0.0,0.0078,0.0001,0.0479,0.0106,0.07639
101,0.0834,4.0,0.0000,0.0000,0.0216,0.0000,0.0000,0.0000,0.0000,0.0,0.0154,0.0010,0.0490,0.0392,0.01041
69,0.1071,0.0,0.0000,0.0000,0.0974,0.0000,0.0000,0.0000,0.0009,0.0,0.0547,0.0000,0.0500,0.0009,0.09375
118,0.1079,5.0,-0.2500,0.0000,0.0042,0.0000,0.0000,0.0000,0.0000,0.0,0.0156,0.0009,0.0554,0.0598,0.00740
102,0.0000,1.0,0.0000,0.0000,0.0203,0.0000,0.0000,0.0000,0.0009,0.0,0.0619,0.0000,0.0765,0.0077,0.01201
8,0.7500,0.0,0.0000,0.0000,0.3783,0.0000,0.0000,0.0000,0.8991,0.0,0.4380,0.0000,0.1155,-0.0041,0.66043
73,0.1944,4.0,0.2500,0.0000,0.1466,0.0000,0.0000,0.0000,0.0009,0.0,0.0664,0.0158,0.1393,0.0930,0.22548
99,0.5000,7.0,0.0000,0.0000,0.1278,0.0133,0.0000,0.0177,0.8919,0.0,0.4377,0.0013,0.1691,0.0698,0.28299
25,0.2500,2.0,0.2222,0.1111,0.1877,0.0100,0.1111,0.0144,0.0981,0.0,0.3146,0.0001,0.2286,0.0174,0.18436


In [10]:
# (comp_df - base_query)['recip_rank'].plot.bar()

In [11]:
def plot_len_correlation(queries, df, metrics):
    lens = {k: len(queries[k]['topic'].split()) for k in queries}
    df['lens'] = df.index.map(lens)
    
    fig = plt.figure() 
    ax = fig.add_subplot(111)
    fig.set_size_inches(16, 4)
    for m in metrics.keys():
        sns.regplot(x=df['lens'], y=df[m], ax=ax, truncate=False)
    ax.set_xlabel('Query length', fontsize=15)
    ax.set_ylabel('Effectiveness $\Delta$', fontsize=15)
    ax.set_ylim(-1, 1)
    ax.tick_params(labelsize=12)
    ax.legend(list(metrics.values()), frameon=True, fontsize=12).get_frame().set_edgecolor("black")
    
    return fig

# def load_queries(path: str):  
#     queries = {}
#     with open(path) as f:
#         data = json.load(f)
#         for topic in data['topics']:
#             leg_ref = None 
#             if 'legislation_ref' in topic: 
#                 leg_ref = topic['legislation_ref']
#             queries[topic['Id']] = {'topic': topic['Topic']}
#     return queries

    
red_dir = '/home/danlocke/go/src/github.com/dan-locke/phd/query-reduction'
# queries = load_queries(os.path.join(red_dir, 'filtered-phrasestop-ictf-0.80-topics.json'))
comp_df = load_1d_dfs([index_name], qrel_paths, _dir, '{0}-ictf-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, 0.80, 0.80, inc, per_query=True)[0][0]
interped_df = (comp_df - base_query)
len_corr_fig = plot_len_correlation(queries, interped_df, {'err@20': 'ERR@20', 'rbp@0.80': 'RBP'})
len_corr_fig.savefig('figures/ausnl-len-reduction-8.pdf')

# queries = load_queries(os.path.join(red_dir, 'filtered-phrasestop-ictf-0.50-topics.json'))
comp_df = load_1d_dfs([index_name], qrel_paths, _dir, '{0}-ictf-{1:.2f}-topics-{0}-unigram_dir_mu_1050.00.run', rel_levels, 0.50, 0.50, inc, per_query=True)[0][0]
interped_df = (comp_df - base_query)
len_corr_fig = plot_len_correlation(queries, interped_df, {'err@20': 'ERR@20', 'rbp@0.80': 'RBP'})
len_corr_fig.savefig('figures/ausnl-len-reduction-5.pdf')

<Figure size 1152x288 with 1 Axes>

<Figure size 1152x288 with 1 Axes>

In [12]:
def correlation(queries, df, metrics):
    lens = {k: len(queries[k]['topic'].split()) for k in queries}
    df['lens'] = df.index.map(lens)
    out_df = pd.DataFrame()
    out_df['pearson'] = df[['lens', *list(metrics.keys())]].corr(method='pearson')['lens'].T
    out_df['kendall'] = df[['lens', *list(metrics.keys())]].corr(method='kendall')['lens'].T
    return out_df.drop(['lens'], axis='index').rename(metrics, axis='index').round(4).to_latex()
    
    
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
write_table('tables/reduction-correlation', correlation(queries, interped_df, metrics))